In [ ]:
# Use R to analyze the tracking data collected

## Load tidyverse

In [ ]:
# Load the necessary packages
library('tidyverse')

In [ ]:
# Options
options(repr.matrix.max.cols = 15, repr.matrix.max.rows = 20)

## Import and combine all data frames

In [ ]:
folder_path <- "../output/LocationOutputs/"
csv_files <- list.files(path = folder_path, pattern = "\\.csv", full.names = TRUE)

In [ ]:
list_of_dataframes <- lapply(csv_files, read.csv)
#head(list_of_dataframes)

In [ ]:
combined_data <- bind_rows(list_of_dataframes)
#head(combined_data)

In [ ]:
combined_data <- (
    combined_data %>% mutate(File = str_sub(File, end = -5))
)

head(combined_data)

In [ ]:
reference_path <- "../tables/video_id.csv"
reference_csv <- read.csv(reference_path)

In [ ]:
head(reference_csv)

In [ ]:
mastersheet <- inner_join(reference_csv, combined_data, by="File")
# Filter_out outliers
mastersheet <- (
    mastersheet %>% filter (Frame < 20000, Distance_px < 80)
)
#head(mastersheet)

## Plot the bird path using different facets

In [ ]:
#plot <- ggplot(
#    mastersheet, aes(x=X, y=Y, color=Bird)
#) + geom_point()
#
#plot

In [ ]:
options(repr.plot.width=15, repr.plot.height=12)
plot <- (
    ggplot(mastersheet, aes(x=X, y=Y))
    + geom_point()
    + facet_wrap(~ Bird)
)

plot

In [ ]:
options(repr.plot.width=9, repr.plot.height=9)
plot <- (
    ggplot(mastersheet, aes(x=X, y=Y))
    + geom_point()
    + facet_wrap(~ File)
)

p <- (plot + ggtitle("Tracking per video file"))

#pdf("../output/pdf/tracking_per_video_file.pdf")
p
#dev.off()

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)

filter <- (
    mastersheet %>% group_by("Bird")
    %>% filter(Condition == "colony")
)

plot <- (
    ggplot(filter, aes(x=X, y=Y))
    + geom_point()
    + facet_grid(Virus ~ File)
)

p <- (plot + ggtitle("Bird's path depending on injected AAV (colony audio)"))

#pdf("../output/pdf/tracking_per_aav_in_silence.pdf")
p
#dev.off()

In [ ]:
options(repr.plot.width=18, repr.plot.height=7)

filter <- (
    mastersheet %>% filter(Bird == "Rb385")
)

plot <- (
    ggplot(filter, aes(x=X, y=Y))
    + geom_point()
    + facet_grid(. ~ Condition)
)

p <- plot + ggtitle("Ruby_385 path depending on broadcast")

#pdf("../output/pdf/Rb331_track_per_broadcast.pdf")
p 
#dev.off()

## Analyze horizontal speed

The travelled distance in pixel can be found in `Distance_px`. Plot the mean velocity (mean distance/frame) for each bird. Filter only silence condition for now.

In [ ]:
#plot <- (
#    ggplot(mastersheet, aes(x =File, y=Distance_px))
#    + geom_boxplot()
#)
#
#plot

In [ ]:
options(repr.plot.width=18, repr.plot.height=3)

filter <- (
    mastersheet %>% filter(Condition == "colony")
)
plot <- (
    ggplot(filter, aes(x=Frame, y=Distance_px))
    + facet_wrap(File ~ Virus)
    + geom_line()
)

p <- (plot + ggtitle("Distance traveled (px/frame)"))

#pdf("../output/pdf/distance_px_per_video_file_aav.pdf")
p
#dev.off()

In [ ]:
options(repr.plot.width=18, repr.plot.height=4)

filter <- (
    mastersheet %>% filter(Condition == "pre_silence")
    %>% filter(Frame < 20000)
    %>% filter(Distance_px < 80)
)
plot <- (
    ggplot(filter, aes(x=Frame, y=Distance_px))
    + facet_wrap(File ~ Virus)
    + geom_line()
)

p <- (plot + ggtitle("Distance traveled (px/frame)"))

#pdf("../output/pdf/distance_px_per_video_file_aav.pdf")
p
#dev.off()

### Make it shiny

Goal: loop through birds and make one line with pre, during and post broacast as well as the corresponding line plot for each of them.

In [ ]:
# list of the variables
names(mastersheet)

In [ ]:
# Define the variables to loop through
analysis_vars <- c("Rb383", "Rb384", "Rb385")
# Removed from the analysis "Rb318", "Rb331", "Rb334", 

# loop through analysis_vars
for (var_name in analysis_vars) {
    #print(paste("Processing variable:", var_name))
    filter <- (
        mastersheet %>% filter(Bird == var_name)
        %>% filter(Frame < 20000, Distance_px < 80)
    )
    options(repr.plot.width=18, repr.plot.height=2)
    plot <- (
        ggplot(filter, aes(x=Frame, y=Distance_px))
        + geom_point()
        + facet_grid(. ~ File)
        #+ default.theme
        )
    p2 <- (plot + ggtitle(paste(var_name, " distance travelled depending on broadcast")))
    print(p2)
}

In [ ]:
# loop through analysis_vars
for (var_name in analysis_vars) {
    #print(paste("Processing variable:", var_name))
    filter <- (
        mastersheet %>% filter(Bird == var_name)
        %>% filter(Frame < 20000, Distance_px < 80)
    )
    #print(head(filter))
    options(repr.plot.width=18, repr.plot.height=7)
    p <- (
        ggplot(filter, aes(x=X, y=Y))
        + geom_point()
        + facet_grid(. ~ File)
        #+ my.theme
        + ggtitle(paste(var_name, " path depending on broadcast"))
    )
    print(p)
}